In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
np.random.seed(0)
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')

In [3]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X.values
X2_val = X_val.values

In [4]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (1-y1) * Y
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (1-y1_val) * Y_val
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [5]:
from fit9_nn import fit, comb_error, mae_error, hrat_error

In [6]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias=True)
        self.fc12 = nn.Linear(24, 1, bias=True)
        
        self.fc21 = nn.Linear(16, 24)
        self.fc22 = nn.Linear(24, 5)
        self.fc23 = nn.Linear(5, 1)
        
        self.fc31 = nn.Linear(2,2, bias=False)
        
    def disc_activation(self, h):
        v1 = torch.sigmoid(h[:,0])
        v2 = torch.exp(h[:,1])
        return torch.stack((v1, v2), dim=1)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)
        
        x2 = self.fc21(x)
        x2 = F.relu(self.fc22(x2))
        x2 = self.fc23(x2)              
        
        x3 = torch.cat((x1, x2), 1)
        x3 = self.fc31(x3)
        x3 = self.disc_activation(x3)
        return x3

In [12]:
n_epochs = 200
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
sig2 = 2 * 950000
PATH = '..\\Data\\checkpoints\\mytraining4.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [14]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [15]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        if m.bias is not None:
            m.bias.data.data.uniform_(-y, y)

In [17]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'Data\\checkpoints\\mytraining4.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, val_error = mae_error,
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 1.0699283579985301, Validation Loss: 289.31280517578125
Epoch 2: Training Loss: 1.0617698033650715, Validation Loss: 289.2580871582031
Epoch 3: Training Loss: 1.0538800259431202, Validation Loss: 289.19976806640625
Epoch 4: Training Loss: 1.046254316965739, Validation Loss: 289.13775634765625
Epoch 5: Training Loss: 1.0388924876848857, Validation Loss: 289.0717468261719
Epoch 6: Training Loss: 1.0317866802215576, Validation Loss: 289.0019836425781
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 1.024937778711319, Validation Loss: 288.928466796875
Epoch 8: Training Loss: 1.0183408856391907, Validation Loss: 288.8507080078125
Epoch 9: Training Loss: 1.011992186307907, Validation Loss: 288.7681884765625
Epoch 10: Training Loss: 1.0058878759543102, Validation Loss: 288.6801452636719
Epoch 11: Training Loss: 1.000024328629176, Validation Loss: 288.5866394042969
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.9943917294343313, Validation Loss: 288.487

Epoch 97: Training Loss: 0.8663612604141235, Validation Loss: 277.9365539550781
Epoch 98: Training Loss: 0.8657237490018209, Validation Loss: 277.76385498046875
Epoch 99: Training Loss: 0.8650891681512197, Validation Loss: 277.5879821777344
Epoch 100: Training Loss: 0.8644579450289408, Validation Loss: 277.4065246582031
Epoch 101: Training Loss: 0.8638304273287455, Validation Loss: 277.2207336425781
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.8632067839304606, Validation Loss: 277.0285339355469
Epoch 103: Training Loss: 0.8625865976015726, Validation Loss: 276.8305358886719
Epoch 104: Training Loss: 0.8619695007801056, Validation Loss: 276.6265563964844
Epoch 105: Training Loss: 0.8613552351792654, Validation Loss: 276.41558837890625
Epoch 106: Training Loss: 0.8607438405354818, Validation Loss: 276.1986999511719
New Checkpoint Saved into PATH
Epoch 107: Training Loss: 0.8601352969805399, Validation Loss: 275.9774475097656
Epoch 108: Training Loss: 0.8595296541849772, Va

Epoch 195: Training Loss: 0.80482945839564, Validation Loss: 279.0180358886719
Epoch 196: Training Loss: 0.8045957485834757, Validation Loss: 278.9983825683594
Epoch 197: Training Loss: 0.8043704231580099, Validation Loss: 278.9810791015625
Epoch 198: Training Loss: 0.8041510681311289, Validation Loss: 278.9568176269531
Epoch 199: Training Loss: 0.8039423227310181, Validation Loss: 278.9259338378906
Epoch 200: Training Loss: 0.8037398954232534, Validation Loss: 278.88995361328125
